# ニューラルネットワークとロジスティック回帰の比較

<a href="http://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn-neural-network-mlpclassifier"><b>sklearn.neural_network.MLPClassifier</b></a> を使用します。

プロダクションのロジスティック回帰との比較を行います。

あわせて、パラメータ値（レイヤー数／ユニット数）を変えて確認を行っております。

## (0) 結論

ロジスティック回帰に比して、accuracy があまり変わらない印象です。

| dataset | MLPClassifier(ベストスコア) | <> | LogisticRegression |
| --- | :---: | :---: | :---: |
| test_daikin_conversation.csv | 0.986703347088（条件1-1） | < | 0.987161852361 |
| test_benefitone_conversation.csv | 0.985436893204（条件1-1） | > | 0.981553398058 |
| test_septeni_conversation.csv | 0.974074074074（条件1） | = | 0.974074074074 |
| test_ptna_conversation.csv | 0.982471516214（条件1-1） | > | 0.973707274321 |


### 下記はご参考

#### (1) MLPClassifier の実行条件とレポートのリンク

| 条件 | レイヤー数 | レイヤーごとのユニット数 | レポートのリンク |
| :---: | :---: | :---: | --- |
| 条件1 | 1層 | 100 | <b><a href="04.ipynb">04.ipynb</a></b> |
| 条件1-2 | 1層 | 200 | <b><a href="04.ipynb">04-200.ipynb</a></b> |
| 条件2 | 2層 | 100 | <b><a href="04-2-layers.ipynb">04-2-layers.ipynb</a></b> |
| 条件2-2 | 2層 | 200 | <b><a href="04-2-layers-200.ipynb">04-2-layers-200.ipynb</a></b> |
| 条件3 | 3層 | 100 | <b><a href="04-3-layers.ipynb">04-3-layers.ipynb</a></b> |
| 条件4 | 4層 | 200->100->50->25 | <b><a href="04-4-layers-var.ipynb">04-4-layers-var.ipynb</a></b> |


####  (2) 実行条件ごとの accuracy 比較
 
　レイヤー数＝1層（条件1 or 条件1-1）が最もスコアが良いように感じられます。

　また、条件が複雑になればなるほど、accuracy が劣化する傾向があるようです。


| dataset | 条件1 | 条件1-1 | 条件2 | 条件2-2 | 条件3 | 条件4 |
| --- | :---: | :---: | :---: | :---: | :---: | :---: |
| test_daikin_conversation.csv | 0.985557083906 | 0.986703347088 |0.983264557542 | 0.980513525906 | 0.95346171481 | 0.943603851444 |
| test_benefitone_conversation.csv | 0.985436893204 | 0.985436893204 | 0.985436893204 | 0.984466019417 | 0.954368932039 | 0.633980582524 |
| test_septeni_conversation.csv | 0.974074074074 | 0.953703703704 | 0.966666666667 | 0.966666666667 | 0.961111111111 | 0.961111111111 |
| test_ptna_conversation.csv | 0.978965819457 | 0.982471516214 | 0.976336546889 | 0.976336546889 | 0.969325153374 | 0.775635407537 |






## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

### (1-1) テストデータをコピー

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
    'test_benefitone_conversation.csv',
    'test_septeni_conversation.csv',
    'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


## (2) ロジスティック回帰でテスト

上記４本のファイル別に、モデル選択(GridSearchCV.fit)--->評価(Evaluator.evaluate)の流れで実行させ、実行時間計測、およびaccuracy計測を行います。

In [3]:
from time import time
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

from learning.core.evaluator import Evaluator
    
def fit_and_cross_validation(path):
    '''
        訓練データのTF-IDFベクターを作成
    '''
    basename = os.path.basename(path)
    print("prepare_tf_idf_vectors: dataset=%s..." % basename)
    t0 = time()

    X, y, vectorizer = TestTool.prepare_tf_idf_vectors(path)
    print("prepare_tf_idf_vectors: done in %0.3fs." % (time() - t0))

    '''
        訓練データ全体を使用して学習実施
        プロダクションと同様、ハイパーパラメータの選択のために
        グリッドサーチを使用します
    '''
    print("GridSearchCV: search and fitting...")
    t0 = time()

    params = {'C': [10, 100, 140, 200]}
    grid = GridSearchCV(LogisticRegression(), param_grid=params)
    grid.fit(X, y)
    estimator = grid.best_estimator_
    print("GridSearchCV: done in %0.3fs." % (time() - t0))

    ''' 
        クロスバリデーション（モデル評価フェーズ）を実施
        プロダクションと同様、Evaluator クラスを使用して評価します
        
        Evaluator クラスで使用している cross_val_score 関数は、
        引数の estimator により、
        内部で fit, predict, predict_proba の各関数を実行しています。
    '''
    print("Evaluator: evaluating...")
    t0 = time()

    evaluator = Evaluator()
    evaluator.evaluate(estimator, X, y, threshold=0.5)
    print("Evaluator: done in %0.3fs." % (time() - t0))
    
    return (basename, X, y, vectorizer, estimator, evaluator)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
list_of_classifiers = []
for path in temp_path:
    classifier = fit_and_cross_validation(path)
    list_of_classifiers.append(classifier)

prepare_tf_idf_vectors: dataset=test_daikin_conversation.csv...


2017/04/04 PM 01:19:51 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/04/04 PM 01:19:51 TextArray#__init__ start
2017/04/04 PM 01:20:01 TextArray#to_vec start
2017/04/04 PM 01:20:01 TextArray#to_vec end
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


prepare_tf_idf_vectors: done in 10.615s.
GridSearchCV: search and fitting...


2017/04/04 PM 01:23:32 self.threshold: 0.5


GridSearchCV: done in 210.984s.
Evaluator: evaluating...


...

prepare_tf_idf_vectors: done in 1.867s.
GridSearchCV: search and fitting...


2017/04/04 PM 01:24:19 self.threshold: 0.5


GridSearchCV: done in 4.521s.
Evaluator: evaluating...


2017/04/04 PM 01:24:19 Evaluator#evaluate#elapsed time: 489.430904 ms
2017/04/04 PM 01:24:19 accuracy: 0.973707274321


0.973707274321
Evaluator: done in 0.492s.
